# Uniswap V3 Subgraph Liquidity Query Example

This notebook is just the `.ipynb` (for easier reading of both code and output on github) implementation of [this code](https://github.com/atiselsts/uniswap-v3-liquidity-math/blob/master/subgraph-liquidity-query-example.py) (credit: https://github.com/atiselsts) with slight modifications, showing an example on how to query the Uniswap v3 Subgraph information to show the amounts locked in the current tick range of the USDC/WETH 0.3% pool.

See the technical note [Liquidity Math in Uniswap v3](http://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf), [Uniswap v3 whitepaper](https://uniswap.org/whitepaper-v3.pdf), and [Uniswap V3 Subgraph](https://thegraph.com/legacy-explorer/subgraph/uniswap/uniswap-v3) for more details.

In [ ]:
# Example that prints the current virtual amounts of assets in the 0.3% USDC/ETH pool
# using liquidity data from the Uniswap v3 subgraph.

import json, math
import urllib.request

# Look at the USDC/ETH 0.3% pool
POOL_ID = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'

# If you want to test with a smaller pool:
# 0.05% GUSD/DAI pool (small pool)
#POOL_ID = "0x7cf12cef5ce9e5e068ebdef470ff8295e26c47b9"

URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

In [ ]:
# GraphQL query to get the pool information
query = """query pools($pool_id: ID!) {
  pools (where: {id: $pool_id}) {
    tick
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""

block_query = """query {
  _meta {
    block {
      number
    }
  }
}"""

In [ ]:
TICK_BASE = 1.0001

# Convert Uniswap v3 tick to a price (i.e. the ratio between the amounts of tokens: token1/token0)
def tick_to_price(tick):
    return TICK_BASE ** tick

# Not all ticks can be initialized. Tick spacing is determined by the pool's fee tier.
def fee_tier_to_tick_spacing(fee_tier):
    return {
        500: 10,
        3000: 60,
        10000: 200
    }.get(fee_tier, 60)

In [ ]:
# Query the subgraph
req = urllib.request.Request(URL)
req.add_header('Content-Type', 'application/json; charset=utf-8')
jsondata = {"query" : block_query}
jsondataasbytes = json.dumps(jsondata).encode('utf-8')
req.add_header('Content-Length', len(jsondataasbytes))
response = urllib.request.urlopen(req, jsondataasbytes)
assert response.status == 200, "Bad response"

In [ ]:
obj = json.load(response)
blocknum = obj['data']['_meta']['block']['number']
print("Ethereum block height {}".format(blocknum))

Ethereum block height 13338936


In [ ]:
# Query the subgraph
req = urllib.request.Request(URL)
req.add_header('Content-Type', 'application/json; charset=utf-8')
jsondata = {"query": query, "variables": {"pool_id": POOL_ID}}
jsondataasbytes = json.dumps(jsondata).encode('utf-8')
req.add_header('Content-Length', len(jsondataasbytes))
response = urllib.request.urlopen(req, jsondataasbytes)
assert response.status == 200, "Bad response"

In [ ]:
obj = json.load(response)
pool = obj['data']['pools'][0]
pool

{'feeTier': '3000',
 'liquidity': '11703237430202825115',
 'tick': '195370',
 'token0': {'decimals': '6', 'symbol': 'USDC'},
 'token1': {'decimals': '18', 'symbol': 'WETH'}}

In [ ]:
# Extract liquidity from the response
L = int(pool["liquidity"])
tick = int(pool["tick"])
ft = int(pool["feeTier"])
tick_spacing = fee_tier_to_tick_spacing(ft)

print("L={}".format(L))
print("tick={}".format(tick))
print("tick spacing={} for fee-tier {}%".format(tick_spacing, ft/1e4))

L=11703237430202825115
tick=195370
tick spacing=60 for fee-tier 0.3%


In [ ]:
token0 = pool["token0"]["symbol"]
token1 = pool["token1"]["symbol"]
decimals0 = int(pool["token0"]["decimals"]) # USDC has 6 decimals
decimals1 = int(pool["token1"]["decimals"]) # WETH has 18 decimals

print("token0 = {} ({} decimals); token1 = {} ({} decimals)".format(token0, decimals0, token1, decimals1))

token0 = USDC (6 decimals); token1 = WETH (18 decimals)


In [ ]:
# Compute the tick range containing current price tick.
# This code would work as well in Python: `tick // TICK_SPACING * TICK_SPACING`
# However, using floor() is more portable.
bottom_tick = math.floor(tick / tick_spacing) * tick_spacing
top_tick = bottom_tick + tick_spacing

# Compute the current price and adjust it to a human-readable format
price = tick_to_price(tick)
adjusted_price = price / (10 ** (decimals1 - decimals0))

# Compute square roots of prices corresponding to the bottom and top ticks
sa = tick_to_price(bottom_tick // 2)
sb = tick_to_price(top_tick // 2)
sp = price ** 0.5

# Compute real amounts of the two assets
amount0 = L * (sb - sp) / (sp * sb) # eq(4) in technical note
amount1 = L * (sp - sa) # eq(9) in technical note

# Adjust them to a human-readable format
adjusted_amount0 = amount0 / (10 ** decimals0)
adjusted_amount1 = amount1 / (10 ** decimals1)

print("Current price: {:.6g} {} for 1 {} ({:.6g} {} for 1 {})".format(
    adjusted_price, token1, token0, 1 / adjusted_price, token0, token1))

print("Amounts at the current tick range: {:.2f} {} or {:.2f} {}".format(
    adjusted_amount0, token0, adjusted_amount1, token1))
print("As at Ethereum block height {}".format(blocknum))

Current price: 0.000305061 WETH for 1 USDC (3278.03 USDC for 1 WETH)
Amounts at the current tick range: 1672969.57 USDC or 102.17 WETH
As at Ethereum block height 13338936
